System crashed-Finance Distilbert

In [5]:
!pip install transformers

In [4]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score
from transformers import DistilBertModel, DistilBertTokenizer


In [7]:
# Load your dataset
dataset = pd.read_csv('/content/finance.csv')  # Replace with the path to your dataset
dataset.head()



,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [8]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dataset['Sentence'], dataset['Sentiment'], test_size=0.2, random_state=42)

In [9]:
# Initialize the DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
# Tokenize and encode the text data using DistilBERT tokenizer
X_train_encoded = tokenizer.batch_encode_plus(
    X_train.tolist(),
    padding=True,
    truncation=True,
    max_length=256,
    return_tensors='pt'
)

In [11]:
X_test_encoded = tokenizer.batch_encode_plus(
    X_test.tolist(),
    padding=True,
    truncation=True,
    max_length=256,
    return_tensors='pt'
)

In [ ]:
import torch
# Extract the embeddings from DistilBERT
with torch.no_grad():
    train_embeddings = model(input_ids=X_train_encoded['input_ids'], attention_mask=X_train_encoded['attention_mask'])
    test_embeddings = model(input_ids=X_test_encoded['input_ids'], attention_mask=X_test_encoded['attention_mask'])

# this code cant be used since it raise out of memeory issue


In [12]:
import torch

batch_size = 32  # Set your desired batch size

# Get the total number of samples
num_samples = len(X_train_encoded['input_ids'])

# Calculate the total number of batches
num_batches = (num_samples + batch_size - 1) // batch_size

# Create empty lists to store the embeddings
train_embeddings = []
test_embeddings = []


In [18]:
print("X_train_encoded:")
print(X_train_encoded)

print("X_test_encoded:")
print(X_test_encoded)


X_train_encoded:
{'input_ids': tensor([[  101,  1996,  2723,  ...,     0,     0,     0],
        [  101,  2053,  9430,  ...,     0,     0,     0],
        [  101, 19387,  1030,  ...,     0,     0,     0],
        ...,
        [  101,  1002, 13675,  ...,     0,     0,     0],
        [  101, 18731,  9598,  ...,     0,     0,     0],
        [  101,  8915,  9363,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
X_test_encoded:
{'input_ids': tensor([[  101,  1037,  2261,  ...,     0,     0,     0],
        [  101, 12435,  5658,  ...,     0,     0,     0],
        [  101, 26060,  2003,  ...,     0,     0,     0],
        ...,
        [  101,  1052, 12273,  ...,     0,     0,     0],
        [  101,  1996,  2194,  ...,     0,     0,     0],
        [  101,  2000,  2156,  ..., 

In [23]:
# Iterate over the training data in batches
for i in range(0, num_samples, batch_size):
    input_ids_batch = X_train_encoded['input_ids'][i:i+batch_size]
    attention_mask_batch = X_train_encoded['attention_mask'][i:i+batch_size]

    # Extract embeddings for the current batch
    with torch.no_grad():
        train_batch_embeddings = model(input_ids=input_ids_batch, attention_mask=attention_mask_batch).last_hidden_state

    # Append the batch embeddings to the list
    train_embeddings.append(train_batch_embeddings)

# Concatenate the embeddings along the batch axis
train_embeddings = torch.cat(train_embeddings, dim=0)

# Iterate over the testing data in batches
num_samples_test = len(X_test_encoded['input_ids'])
for i in range(0, num_samples_test, batch_size):
    input_ids_batch = X_test_encoded['input_ids'][i:i+batch_size]
    attention_mask_batch = X_test_encoded['attention_mask'][i:i+batch_size]

    # Extract embeddings for the current batch
    with torch.no_grad():
        test_batch_embeddings = model(input_ids=input_ids_batch, attention_mask=attention_mask_batch).last_hidden_state

    # Append the batch embeddings to the list
    test_embeddings.append(test_batch_embeddings)

# Concatenate the embeddings along the batch axis
test_embeddings = torch.cat(test_embeddings, dim=0)

# Check the sizes of train_embeddings and test_embeddings
print("Train Embeddings Size:", train_embeddings.size())
print("Test Embeddings Size:", test_embeddings.size())


Train Embeddings Size: torch.Size([7041, 150, 768])
Test Embeddings Size: torch.Size([3507, 93, 768])


In [25]:

# Flatten the embeddings to 2D shape (num_samples, embedding_size)
train_embeddings_flat = train_embeddings.view(train_embeddings.size(0), -1)
test_embeddings_flat = test_embeddings.view(test_embeddings.size(0), -1)


In [26]:
from sklearn.svm import SVC

In [ ]:
# Create an SVM classifier
svm_classifier = SVC()

# Fit the SVM classifier on the training embeddings and labels
svm_classifier.fit(train_embeddings_flat, y_train)

# Predict the labels for the test embeddings
predicted_labels = svm_classifier.predict(test_embeddings_flat)

# Evaluate the performance of the classifier (e.g., accuracy)
accuracy = (predicted_labels == y_test).mean()
print("Accuracy:", accuracy)